## БИБЛЫ

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import torch
import numpy as np

if torch.cuda.is_available():
    from torch.cuda import FloatTensor, LongTensor
    DEVICE = torch.device('cuda')
else:
    from torch import FloatTensor, LongTensor
    DEVICE = torch.device('cpu')

np.random.seed(313)

In [4]:
torch.torch_version.internal_version

'1.13.0+cu116'

In [6]:
import pandas as pd
from pymongo import MongoClient
from datetime import datetime
import re

## Загрузка данных

In [7]:
CONNECTION_STRING = ""

In [8]:
def atoi(text):
    return int(text) if text.isdigit() else text


def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]


def get_latest_preprocessed_depression_df(do_remove_id=True, do_remove_age=True):
    client = MongoClient(CONNECTION_STRING)
    print(client.list_database_names())
    db = client['depression']
    print(db.list_collection_names())
    preprocessed_collections = [coll for coll in db.list_collection_names() if 'preprocessed_depression_data' in coll]
    preprocessed_collections.sort(key=natural_keys)
    latest_collection_name = 'preprocessed_depression_data_10_12_2022_17_01_54'
    latest_preprocessed_data_collection = db[latest_collection_name]
    preprocessed_data_cursor = latest_preprocessed_data_collection.find()
    dataframe = pd.DataFrame(list(preprocessed_data_cursor))
    if do_remove_id:
        del dataframe['_id']
    if do_remove_age:
        del dataframe['age']
    print(dataframe)
    return dataframe

depression_df = get_latest_preprocessed_depression_df()

['depression']
['preprocessed_depression_data_10_12_2022_17_01_54', 'raw_depression_data', 'preprocessed_depression_data_19_11_2022_13_24_38', 'depression_data_accessors', 'users']
                                                    text  label
0      когда то я был добрым романтиком который стрем...      1
1      у меня проблемы с девушкой каждую ссору я не м...      1
2      вся моя жизнь это один сплошной ад в котором я...      1
3      я хочу уснуть и не проснуться каждый день одно...      1
4      здраствуйте я каждый день просыпаюсь с мыслью ...      1
...                                                  ...    ...
62553  в минувшие выходные в спортблоке калмгу прошли...      0
62554  v сезон club129270552 международной детской не...      0
62555  втб подвел итоги первого корпоративного акселе...      0
62556  втб запустит бесплатное пополнение карт других...      0
62557  в сызрани вновь пройдет ярмарка вакансий для р...      0

[62558 rows x 2 columns]


##Разделение на train, test

In [9]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(depression_df, test_size=0.20, random_state=313, stratify=depression_df['label'])

print(train_df)
print(test_df)

                                                    text  label
25918  пожалуйста что то тоска одолевает поговорите с...      1
9268   здраствуйте не знаю с чего начать наверное это...      1
12399  очень очень больно об этом рассказывать мне пл...      1
53014  валенсия испания город валенсия расположен на ...      0
59164  в toyota размышляют о новом компактном кроссов...      0
...                                                  ...    ...
55793  в нью йорке дебютировал компактный кроссовер l...      0
21704  я потерял смысл существования хотя всеми силам...      1
1711   здравствуйте приш л сюда из форума где пару дн...      1
23600  прошу всех вас о помощи о совете уже было мног...      1
36983  by transport продам пежо 307 универсал 2004 2 ...      0

[50046 rows x 2 columns]
                                                    text  label
46479  сегодня club109344729 волонт ры победы провели...      0
27915  люди которые шутят о менструальной болезни чре...      1
60853  историк

## Преобразование к нужному для обработки виду

In [10]:
from datasets import Dataset

ds_train = Dataset.from_pandas(train_df)
ds_test = Dataset.from_pandas(test_df)

## Токенизация

In [11]:
from transformers import AutoTokenizer, DataCollatorWithPadding

tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/distilrubert-tiny-cased-conversational")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def tokenize(dataset):
    return tokenizer(dataset["text"], truncation=True, max_length=512, padding='max_length')

In [15]:
tokenized_ds_train = ds_train.map(tokenize, batched=True)
tokenized_ds_test = ds_test.map(tokenize, batched=True)
print(tokenized_ds_train)

  0%|          | 0/51 [00:00<?, ?ba/s]

  0%|          | 0/13 [00:00<?, ?ba/s]

Dataset({
    features: ['text', 'label', '__index_level_0__', 'input_ids', 'attention_mask'],
    num_rows: 50046
})


## Оценщик

In [12]:
import evaluate
import numpy as np

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)


## Классификатор депрессии

In [13]:
id2label = {"0": "NON-DEPRESSIVE", "1": "DEPRESSIVE"}
label2id = {"NON-DEPRESSIVE": "0", "DEPRESSIVE": "1"}

In [17]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "DeepPavlov/distilrubert-tiny-cased-conversational", num_labels=2, id2label=id2label, label2id=label2id
)

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Colab Notebooks/dd-bert-tiny-ep1-b16",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds_train,
    eval_dataset=tokenized_ds_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--DeepPavlov--distilrubert-tiny-cased-conversational/snapshots/a70de709fe33d6879bd82337162bdd2ea19442bd/config.json
Model config DistilBertConfig {
  "_name_or_path": "DeepPavlov/distilrubert-tiny-cased-conversational",
  "activation": "gelu",
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "NON-DEPRESSIVE",
    "1": "DEPRESSIVE"
  },
  "initializer_range": 0.02,
  "label2id": {
    "DEPRESSIVE": "1",
    "NON-DEPRESSIVE": "0"
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 2,
  "output_attentions": true,
  "output_hidden_states": true,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.25.1",
  "vocab_size": 119547
}

loading weights file pytorch_model.bin from 

Epoch,Training Loss,Validation Loss


ValueError: ignored

In [ ]:
torch.save(model, '/content/drive/MyDrive/Colab Notebooks/dd-bert-tiny-ep1-b16')

In [ ]:
loaded_model = torch.load('/content/drive/MyDrive/Colab Notebooks/dd-bert-tiny-ep1-b16')